### PromptTemplate 
* [PromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html#langchain_core.prompts.prompt.PromptTemplate)
* [ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate)
* [ChatMessagePromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatMessagePromptTemplate.html#langchain_core.prompts.chat.ChatMessagePromptTemplate)
* [FewShotPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.few_shot.FewShotPromptTemplate.html#langchain_core.prompts.few_shot.FewShotPromptTemplate)
* PartialPrompt

In [1]:
# poetry add python-dotenv langchain langchain-openai

In [2]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_3


##### 1) PromptTemplate 의 from_template() 함수 사용
* 주로 LLM(텍스트 완성형 모델, ex. Ollama, GPT-3.5)과 함께 사용
* 하나의 문자열 프롬프트를 생성

In [11]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pprint import pprint

template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    #model="openai/gpt-oss-120b",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

chain = prompt_template | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3})
pprint(response)

('ChatGPT는 인터넷에서 수집한 방대한 텍스트를 바탕으로 단어들이 나올 확률을 학습합니다.  \n'
 '학습 과정에서 사람의 도움을 받아 더 나은 답을 선택하도록 보상을 주는 강화학습 방식을 사용합니다.  \n'
 '결과적으로 입력된 질문에 가장 적절한 단어들을 순차적으로 예측해 문장을 만들어 답변합니다.')


##### 2) PromptTemplate 결합하기
* 동일한 Prompt 패턴을 사용하지만 여러 개의 질문을 작성해서 LLM을 실행할 수도 있습니다.

In [12]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="한국어")
print(combined_prompt)

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"한국어"})

pprint(response)

input_variables=['count', 'language', 'model_name'] input_types={} partial_variables={} template='{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요.\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요.'
('ChatGPT는 인터넷의 방대한 텍스트를 바탕으로 다음 토큰을 예측하도록 사전학습(pre-training)한 뒤, 인간의 대화 데이터로 '
 '미세조정(fine-tuning)해 질문에 자연스럽고 유용하게 답하도록 만듭니다.  \n'
 '이 과정에서 트랜스포머 아키텍처의 어텐션 메커니즘이 문맥을 기억·활용하게 되며, 강화학습 기반의 RLHF가 사람의 선호도를 반영해 '
 '안전성과 정확성을 높입니다.  \n'
 '결국 통계적 패턴 매칭으로 사고나 의식 없이 단어 확률을 계산해 문장을 생성하는 것이 핵심 원리입니다.\n'
 '\n'
 'ChatGPT 모델의 장점 요약\n'
 '- 문맥 이해·유지가 뛰어나 긴 대화·복잡한 지시도 자연스럽게 처리  \n'
 '- 다국어·다분야(코딩, 번역, 요약, 창작 등)에 즉시 활용 가능한 범용성  \n'
 '- 별도 학습 없이 프롬프트만으로 빠르게 맞춤형 답변 생성  \n'
 '- 인간 피드백 강화학습(RLHF)로 유해·편향 출력을 줄여 안전성↑  \n'
 '- API·플러그인·자동화 툴과 쉽게 연결돼 생산성 도구·서비스 통합이 간편\n'
 '\n'
 'ChatGPT와 비슷한 AI 모델(한국어 명칭)\n'
 '- 구글 젬마이(Gemini)\n'
 '- 안트로픽 클라우드(Claude)\n'
 '- 메타 라마(Llama)\n'
 '- 마이크로소프트 코파일럿(Copilot)')


#### PromptTemplate 의 파라미터를 배열 형태로 하여 여러개 사용하는 경우

In [16]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 3},
    {"model_name": "Gemini", "count": 4},
    {"model_name": "Claude", "count": 4}
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions] #파이썬에서 파라미터로 딕셔너리 받을 때 아스타(*) 2개 씀
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

for prompt in formatted_prompts:
    print(type(prompt), prompt)
    response = llm.invoke(prompt)
    pprint(response.content)

['GPT-4 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.', 'Gemini 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.', 'Claude 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.']
<class 'str'> GPT-4 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.
('GPT-4는 방대한 텍스트 데이터에서 다음 토큰을 예측하도록 훈련된 트랜스포머 기반 언어 모델입니다.  \n'
 '학습 과정에서는 인간의 피드백을 반영한 강화학습(RLHF)을 추가로 적용해 유용하고 안전한 응답을 만들어냅니다.  \n'
 '결국 통계적 패턴을 학습해 주어진 문맥에 가장 적합한 단어를 순차적으로 생성하는 방식으로 작동합니다.')
<class 'str'> Gemini 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Gemini는 대규모 텍스트·코드·이미지·음성·영상을 동시에 학습하는 멀티모달 Transformer로, 시퀀스 전체를 한 번에 처리하는 '
 '‘Attention’ 메커니즘으로 패턴을 익힙니다.  \n'
 '학습 단계에서는 다음 토큰을 맞추는 사전 학습(pre-training)과 인간 피드백 강화학습(RLHF)으로 정렬을 거쳐, 지식과 '
 '안전성을 동시에 높입니다.  \n'
 'Mixture-of-Experts(MoE) 구조로 필요한 파라미터만 활성화해 효율을 높였고, TPU 클러스터에서 대규모 배치·정밀도 '
 '최적화를 반복해 글로벌한 언어·도메인 데이터를 압축합니다.  \n'
 '결과적으로 Gemini는 텍스트 생성뿐 아니라 이미지·영상 이해·생성까지 가능한 통합 모델로, 새로운 입력에 대해 내부 표현을 유연하게 '
 '조합해 답을 만들어냅니다.')
<class 'str'> Claude 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Claude는 방대한 텍스트 데이터를 미리 학습해 언어의 패턴을 학습한 대규모 언어 모델이다.  \n'
 '사용자 입력을 토큰 단위로 잘게 쪼개 이전 토큰들

##### 2) ChatPromptTemplate
* Tuple 형태의 system, user, assistant 메시지 지원
* 여러 개의 메시지를 조합하여 LLM에게 전달 가능
* 간결성과 가독성이 높고 단순한 구조

In [17]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    # role, message
    ("system", "This system is an expert in answering questions about {topic}. Please provide clear and detailed explanations."),
    ("human", "{model_name} 모델의 학습 원리를 설명해 주세요."),
])

messages = chat_prompt.format_messages(topic="AI", model_name="ChatGPT")
print(messages)

# 생성한 메시지를 바로 주입하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

print(type(response))
print(response.content)

[SystemMessage(content='This system is an expert in answering questions about AI. Please provide clear and detailed explanations.', additional_kwargs={}, response_metadata={}), HumanMessage(content='ChatGPT 모델의 학습 원리를 설명해 주세요.', additional_kwargs={}, response_metadata={})]
<class 'langchain_core.messages.ai.AIMessage'>
ChatGPT는 ‘Generative Pre-trained Transformer’의 약자로, 세 단계에 걸쳐 ‘말을 만들어 내는 능력’을 익힌다. 핵심 흐름을 짚어 보면 다음과 같다.

1. Pre-training(사전 학습) – “인터넷 글을 그대로 암기하지 않고, 패턴을 잡아 머릿속에 압축해 넣는 단계”
   • 목표  
     – 다음 단어를 맞추는 “확률 퍼즐”을 엄청 많이 풀면서, ‘언어의 통계적 구조’를 학습한다.  
   • 입력  
     – 웹, 책, 위키 등 대량의 텍스트를 토큰(token, 단어나 부분 단어) 단위로 자른 뒤, 이 토큰 시퀀스를 Transformer 블록에 넣는다.  
   • 모델 구조  
     – 12~96개 층(layer)의 Transformer 디코더 스택.  
     – 셀프 어텐션(self-attention)이 문장 안의 모든 토큰 사이 ‘의존 관계’를 한꺼번에 계산.  
     – Feed-forward 네트워크가 각 토큰의 ‘다음에 나올 법한 토큰 점수’를 출력.  
   • 손실 함수  
     – Cross-entropy: “정답 토큰에 주어진 확률을 1에 가깝게, 나머지는 0에 가깝게.”  
   • 결과  
     – ‘커다란 언어 모델’ 완성. 질문-답, 번역, 요약 등 어떤 텍스트도 ‘자연스럽게 이어 쓰는’ 기초 능력 보유

In [18]:
# 체인을 생성하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain = chat_prompt | llm | StrOutputParser()

response = chain.invoke({"topic":"AI", "model_name":"ChatGPT"})
print(type(response))
print(response)

<class 'str'>
ChatGPT의 학습은 “두 단계”로 압축할 수 있습니다.  
1) ‘언어 모델’로서의 능력을 먼저 길러 놓고,  
2) 사람의 의도를 따르는 방향으로 ‘미세 조정’합니다.  
각 단계에서 쓰이는 데이터, 목적함수, 최적화 기법이 다르며, 모든 학습은 **결국 “다음 토큰을 얼마나 잘 맞히느냐”** 를 기준으로 이뤄집니다.

----------------------------------------
1단계. 사전학습(Pre-training)  
- 목표: 인터넷에 있는 방대한 텍스트를 읽고, 통계적 패턴을 내면화해 일반적인 ‘언어 표현’을 얻는다.  
- 데이터: 웹페이지, 위키, 책, 논문 등 수십~수백 GB(1~2조 토큰).  
- 모델: Transformer 디코더(예: 175B 파라미터 GPT-3).  
- 최적화:  
  – 입력 토큰 시퀀스 x1 … xT 가 주어졌을 때,  
    L(θ) = –Σ log Pθ(xt | x<t) 를 Gradient Descent로 최소화.  
  – 분산·병렬 학습(DDP, ZeRO, FSDP) + 혼합 정밀도(FP16/BF16)로 수천 개 GPU에서 수 주~수개월.  
- 결과: ‘세상의 글’은 거의 다 읽었지만, 사람 질문에 대해 “정직하게” 답하거나, “안전·정책”을 고려하지 않음.  
  → 따라서 2단계가 필요.

----------------------------------------
2단계. 보상 기반 미세 조정(RLHF)  
OpenAI는 2022년 논문 “InstructGPT”에서 소개한 3-단계 절차를 그대로 확장해 ChatGPT에 적용했다.

Step 1. 지도 미세 조정(SFT, Supervised Fine-Tuning)  
- 데이터: 프롬프트(질문, 지시)–응답 쌍 1~10만 개를 인간 라벨러가 작성.  
- 목적: 최대우도(negative log-likelihood) 최소화.  
- 결과: 프롬프트 형식은 맞추지만, 품질·안전·허위 여부는 사람이 매긴 ‘보상 값’을

#### 3) ChatPromptTemplate
* SystemMessagePromptTemplate와 HumanMessagePromptTemplate 클래스 사용
* 객체 지향적 접근 - Message 객체를 독립적으로 생성 가능
* 여러 조건에 따라 다른 시스템 메시지 선택

```python
if user_is_beginner:
    system_message = SystemMessagePromptTemplate.from_template("초보자를 위한 설명: {topic}")
else:
    system_message = SystemMessagePromptTemplate.from_template("전문가를 위한 상세 분석: {topic}")
```

In [20]:
# ChatMessagePromptTemplate 활용

from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    ChatMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "You are an AI expert in {topic}. Please provide clear and detailed explanations."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="What is deep learning?")

# LLM 호출
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

# 결과 출력
print(response.content)

 (The prompt is not asking you to write an essay yourself; it is asking for a clear, detailed explanation.)

Deep learning is a sub-field of machine learning that uses artificial neural networks with many layers (hence “deep”) to learn patterns directly from large amounts of data. Whereas traditional machine-learning models often rely on carefully hand-engineered features, deep-learning models learn the features themselves by adjusting millions (sometimes billions) of numeric parameters called weights. Once trained, these models can map new inputs (images, audio waveforms, text, sensor data, etc.) to desired outputs (class labels, translated sentences, steering angles, etc.) with high accuracy.

Key ideas behind deep learning:

1. Representation learning  
   Each layer in a deep network learns a representation of the data that makes the next layer’s job easier. Early layers usually capture low-level features (e.g., edges in images, phonemes in speech), while deeper layers compose thes

#### ChatMessagePromptTemplate는 여러 종류의 메시지(시스템, 인간, AI)를 조합하여 복잡한 프롬프트를 생성할 때 유용합니다.
* SystemMessagePromptTemplate: 이 템플릿은 AI 모델에게 역할을 부여하거나 전반적인 규칙을 설정하는 시스템 메시지를 만듭니다. 위의 예시에서는 "번역을 도와주는 유용한 도우미"라는 역할을 지정합니다.
* HumanMessagePromptTemplate: 이 템플릿은 사용자의 질문이나 요청을 담는 인간 메시지를 만듭니다. 아래의 예시에서는 번역할 텍스트를 입력받습니다.
* ChatPromptTemplate.from_messages: 이 클래스 메서드는 시스템 메시지, 인간 메시지 등 여러 종류의 MessagePromptTemplate 객체들을 리스트로 받아 하나의 채팅 프롬프트 템플릿으로 통합합니다.
* format_messages: 이 메서드는 정의된 템플릿에 실제 값을 채워 넣어 [SystemMessage, HumanMessage] 형태의 리스트를 반환합니다. 이 리스트는 채팅 모델(Chat Model) 에 바로 전달될 수 있습니다.

In [21]:
# 필요한 라이브러리 임포트
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 1. SystemMessagePromptTemplate와 HumanMessagePromptTemplate 생성
# SystemMessagePromptTemplate는 모델의 페르소나 또는 기본 지침을 설정합니다.
system_template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# HumanMessagePromptTemplate는 사용자로부터 받는 입력 프롬프트를 정의합니다.
human_template = "{text_to_translate}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# 2. ChatPromptTemplate 생성
# 위에서 만든 두 템플릿을 리스트로 묶어 ChatPromptTemplate을 만듭니다.
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# 3. 프롬프트 포맷팅
# chat_prompt_template.format_messages()를 사용하여 최종 메시지 리스트를 생성합니다.
# 이 함수는 딕셔너리 형태의 입력 변수를 받습니다.
formatted_prompt = chat_prompt_template.format_messages(
    input_language="English",
    output_language="Korean",
    text_to_translate="I love programming."
)

# 4. 결과 출력
print(formatted_prompt)

# LLM 호출
response = llm.invoke(formatted_prompt)

# 결과 출력
print(response.content)

[SystemMessage(content='You are a helpful assistant that translates English to Korean.', additional_kwargs={}, response_metadata={}), HumanMessage(content='I love programming.', additional_kwargs={}, response_metadata={})]
나는 프로그래밍을 사랑해요.


##### 4) FewShotPromptTemplate
* FewShotPromptTemplate은 모델이 특정 형식을 따르게 하거나, 일관된 응답을 생성하도록 유도할 때 유용합니다.
* 도메인 지식이 필요하거나, AI가 오답을 줄이고 더 신뢰할 만한 답변을 생성하도록 해야 할 때 효과적입니다.

##### 4-1) PromptTemplate을 사용하지 않는 경우

In [22]:
# PromptTemplate을 사용하지 않는 경우
from langchain_openai import ChatOpenAI

# model
#llm = ChatOpenAI(model="gpt-3.5-turbo")

# chain 실행
result = llm.invoke("태양계의 행성들을 간략히 정리해 주세요.")

print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
태양계 행성(해왕성 안쪽)을 **내행성·외행성**으로 나누어 한 줄씩 정리하면 다음과 같습니다.

1. **수성** – 태양에 가장 가까운 암석형 행성  
2. **금성** – 두꺼운 대권·온실효과로 표면이 가장 뜨거운 행성  
3. **지구** – 유일하게 생명이 확인된 행성  
4. **화성** – ‘붉은 행성’, 얼음·흔적 강물이 있는 암석형 행성  
5. **목성** – 태양계 최대 기체행성, 90개 이상 위성(대표 간선 이오·목위일)  
6. **토성** – 환상적인 고리, 밀도가 물보다 낮음  
7. **천왕성** – 거꾸로 누운 자전축, 고리·위성 27개  
8. **해왕성** – 태양계 최대 폭풍, 달 트리톤은 역행궤도


##### 4-2) FewShotChatMessagePromptTemplate 사용하는 경우

In [27]:
# FewShotChatMessagePromptTemplate 사용하는 경우
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
chain = final_prompt | llm
print(chain)

# 테스트 실행
result = chain.invoke({"input": "양자컴퓨팅에 대하여 설명해 주세요."})
print(result.content)

first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다.'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': '뉴턴의 운동 법칙을 요약해 주세요.', 'output': '### 뉴턴의 운동 법칙\n1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.\n2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.\n3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다.'}, {'input': '지구의 대기 구성 요소를 알려주세요.', 'output': '### 지구 대기의 구성\n- **질소 (78%)**: 대기의 대부분을 차지합니다.\n- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.\n- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.\n- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다.'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[

#### 5-1) PartialPrompt 
* 프롬프트를 더 동적으로 활용할 수 있으며, AI 응답을 더 일관성 있게 조정 가능함

In [29]:
from datetime import datetime
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# 프롬프트 템플릿 정의 (부분 변수 적용)
prompt = PromptTemplate(
    template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}이 맞나요? {season}에 주로 발생하는 지구과학 현상을 3개 알려주세요",
    input_variables=["phenomenon"],  # 사용자 입력 필요
    partial_variables={"season": get_current_season()}  # 동적으로 계절 값 할당
)

# OpenAI 모델 초기화
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

# 특정 계절의 현상 질의
query = prompt.format(phenomenon="태풍 발생")
result = llm.invoke(query)


# 결과 출력
print(f" 프롬프트: {query}")
print(f" 모델 응답: {result.content}")

 프롬프트: 가을에 일어나는 대표적인 지구과학 현상은 태풍 발생이 맞나요? 가을에 주로 발생하는 지구과학 현상을 3개 알려주세요
 모델 응답: 가을에 일어나는 **대표적인 지구과학 현상**은 **태풍 발생**이 맞긴 하지만, **태풍은 여름~가을에 걸쳐 발생**하며 **가을에도 빈번**하게 발생합니다. 따라서 **가을에 주로 발생하거나 뚜렷하게 나타나는 지구과학 현상**을 다음과 같이 정리할 수 있습니다:

---

### ✅ 가을에 주로 발생하거나 뚜렷하게 나타나는 지구과학 현상 3가지

1. **태풍의 고위도 이동 (가을 태풍)**  
   - 여름보다 **고위도(한반도, 일본 등)**로 이동할 확률이 높아짐.  
   - 예: **태풍 링링(2019)**, **태풍 힌남노(2022)** 등.

2. **기압골의 활성화 (활성 기압골)**  
   - **차가운 북서계절풍**과 **따뜻한 남서풍**이 충돌하면서 **기압골이 발달**  
   - 이로 인해 **비가 오고 기온이 급감**하는 **가을철 냉전선** 형성

3. **단풍과 관련된 기온 변화 (일교차 확대)**  
   - **낮과 밤의 기온 차이가 크게 벌어지며**, 이는 **식물의 색소 변화(클로로필→안토시아닌)**를 유도  
   - **지구과학적 관점**에서는 **기온 변화가 생태계에 미치는 영향**으로 볼 수 있음

---

### 🔍 요약
| 현상 | 가을 특징 |
|------|------------|
| **태풍** | 고위도로 이동, 한반도 직접 영향 가능 |
| **기압골** | 냉전선 형성, 비와 급격한 기온 하강 |
| **일교차 확대** | 단풍 유발, 생태계 변화의 지표 |

---

필요하시면 이 중 하나를 더 깊게 설명드릴게요.


In [39]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# Step 1: 현재 계절 결정
season_name = get_current_season()  # 계절 값 얻기
print(f"현재 계절: {season_name}")

현재 계절: 가을


In [40]:

# Step 2: 해당 계절의 자연 현상 추천
prompt2 = ChatPromptTemplate.from_template(
    "{season}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. "
    "각 현상에 대해 간단한 설명을 포함해주세요."
)

# OpenAI 모델 사용
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
##llm = ChatOpenAI(
##    #api_key=OPENAI_API_KEY,
##    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
##    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
##    temperature=0.0
##)

# 체인 2: 자연 현상 추천 (입력: 계절 → 출력: 자연 현상 목록)
chain2 = (
    {"season": lambda x : season_name}  # chain1의 출력을 season 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 실행: 현재 계절에 따른 자연 현상 추천
response = chain2.invoke({})
print(f"\n {season_name}에 발생하는 자연 현상:\n{response}")


 가을에 발생하는 자연 현상:
1.  **성운**: 성운은 가스와 먼지가 많은 천체를 뜻합니다. 가스 성분은 우주에서 가장 가벼운 원소인 수소, 그리고 헬륨입니다. 성운은 별의 생성과 소멸의 과정에 중요한 역할을 합니다. 대표적인 성운으로는 오리온 성운, 카리나 성운, 에타 카리나별 등이 있습니다. 
2.  **은하수**: 밤하늘에서 은하수를 보면 길게 뺨처럼 생긴 띠가 있는 것을 볼 수 있습니다. 이 은하수는 태양계가 속한 은하인 우리 은하의 중심에 있는 별들을 밤하늘에서 바라본 모습입니다. 지구가 속해있는 우리 은하는 나선형 은하로, 밤하늘에서 보이는 은하수는 우리 은하의 중심부 별들은 우리가 중심을 향해 바라본 모습이기 때문에 길게 띠처럼 보이는 것입니다. 
3.  **페르세우스자리 유성우**: 페르세우스자리 유성우는 가을에 주로 관측할 수 있는 유성우입니다. 유성우는 어떤 천체가 지구 대기에 진입하여 불타면서 떨어지는 현상으로 흔히 말하는 별똥별을 의미합니다. 가을밤의 하늘이 맑으면 페르세우스자리 유성우를 한 밤에 최대 50개까지도 관측할 수 있습니다.


#### 5-2) PartialPrompt 
* API 호출 데이터, 시간 정보, 사용자 정보 등을 반영할 때 매우 유용함

In [41]:
import requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

# Partial Prompt 활용
prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)

# LLM 모델 설정
#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# 모델에 프롬프트 전달 및 응답 받기
response = llm.invoke(prompt.format())

# 결과 출력
print(" 프롬프트:", prompt.format())
print(" 모델 응답:", response.content)

 프롬프트: 현재 1달러 = 1377.98원 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.
 모델 응답: ## 2024년 5월 8일 환율 분석

2024년 5월 8일, 1달러는 1377.98원으로 환산됩니다. 이는 2024년 5월 8일 한국은행이 고시한 환율에 따른 정보이며, 실제 거래 시 소매환율(환전 시 적용되는 환율)이 적용됩니다.

* 전일 환율: 1달러 = 1376.7원 
* 상승/하락: 상승 (전일 대비 1.28원 상승) 
* 거래량: (해당 정보는 실시간으로 제공되지 않으므로, 별도로 확인 필요) 

## 환율에 영향을 주는 요인들

1. **경제 지표**: 미국의 비농업 부문 고용 지표, GDP 성장률, 인플레이션율 등 경제 지표는 환율에 큰 영향을 미칩니다. 
2. **글로벌 경제 상황**: 세계 경제의 불확실성, 무역 전쟁, 국제 유가 변동 등도 환율에 영향을 줍니다. 
3. **통화 정책**: 미국 연방준비제도(Fed)의 금리 결정과 한국은행의 통화 정책도 환율 변동에 영향을 미치는 중요한 요인입니다. 
4. **정치적 요인**: 정치적 불확실성이나 지정학적 리스크도 환율에 영향을 줄 수 있습니다.

## 최근 환율 동향

2024년 5월 8일 기준, 원화 가치는 미국 달러 대비 소폭 상승한 것으로 나타납니다. (전일 대비 1.28원 상승한 1377.98원) 

이는 최근 글로벌 경제의 불확실성이 다소 완화되고, 한국 경제에 대한 긍정적인 전망이 반영된 결과일 수 있습니다.

하지만, 환율은 다양한 요인에 의해 변동성이 크므로, 지속적인 모니터링이 필요합니다. 특히, 미국의 통화 정책과 글로벌 경제 상황의 변화는 원화 가치에 큰 영향을 미칠 것입니다.

## 전망

향후 환율 전망은 여러 가지 요인에 따라 달라질 수 있습니다. 
* 미국의 금리 인하 가능성
* 한국의 경제 성장률
* 글로벌 무역 상황

등을 고려해야 합니다.

## 요약

2024년 5월 8일, 1달러는 1377.98원으로 환산되며, 전일 대비 소폭 